In [1]:
from quchem_ibm.Qiskit_Chemistry import *

In [2]:
transformation='BK'

In [3]:
from quchem.Hamiltonian_Generator_Functions import *
from quchem.Graph import *
## HAMILTONIAN start

Molecule = 'LiH'
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., 1.45))]
basis = 'sto-6g'


### Get Hamiltonian
Hamilt = Hamiltonian_PySCF(Molecule,
                     run_scf=1, run_mp2=1, run_cisd=1, run_ccsd=1, run_fci=1,
                     basis=basis,
                     multiplicity=1,
                     geometry=geometry)  # normally None!
QubitHamiltonian = Hamilt.Get_Qubit_Hamiltonian(threshold=None, transformation=transformation)
### HAMILTONIAN end

#####################################

print(QubitHamiltonian)


fci_energy = Hamilt.molecule.fci_energy
print(fci_energy)

(-4.142299396835114+0j) [] +
(0.02971258240931597+0j) [X0 X1 X2] +
(-0.0015779818833572964+0j) [X0 X1 X2 X3 Y7 Y11] +
(0.0009529034207239978+0j) [X0 X1 X2 Y3 Y5] +
(0.001961955629452306+0j) [X0 X1 X2 Z3] +
(0.00293494132824593+0j) [X0 X1 Z2 X3 Y7 Z9 Y10 X11] +
(0.0034056263156493015+0j) [X0 X1 Z2 Y3 Y4 X5] +
(-0.0011759180512875543+0j) [X0 X1 X3 X4 Y7 Y11] +
(0.0011649814581183142+0j) [X0 X1 X3 Y4 Y5 Z6 Z7] +
(0.0011759180512875543+0j) [X0 X1 X3 Y4 Z5 Y7 Z9 Z10 X11] +
(0.0025509806659133943+0j) [X0 X1 X3 Z4 Y5 Y6 Z7] +
(-0.0023168929464120985+0j) [X0 X1 X3 Z4 Z5 Y7 Z9 Y10 X11] +
(-0.0011409748951245443+0j) [X0 X1 X3 Z4 Y7 Z9 Y10 X11] +
(0.0015099541958032668+0j) [X0 X1 X3 X6 Y7 Y11] +
(-0.0015033808265301754+0j) [X0 X1 X3 Z6 Y7 Z9 Y10 X11] +
(0.0015099541958032672+0j) [X0 X1 X3 Y7 X8 Y11] +
(-0.0015099541958032672+0j) [X0 X1 X3 Y7 Y8 Z10 X11] +
(-0.0015033808265301747+0j) [X0 X1 X3 Y7 Z8 Z9 Y10 X11] +
(6.573369273092662e-06+0j) [X0 X1 X3 Y7 Z8 Y10 X11] +
(-0.0007014172134460299+0j) [X0

From PHYS. REV. X, **8**, 031022 (2018):

$$LiH_{BK}^{HartreeFock} = | 101000000000\rangle$$

In [10]:
from quchem.Ansatz_Generator_Functions import *
n_electrons=Hamilt.molecule.n_electrons
n_qubits=Hamilt.molecule.n_qubits

ansatz_obj = Ansatz(n_electrons,n_qubits)
print('JW ground state = ', ansatz_obj.Get_JW_HF_state_in_OCC_basis())
print('BK ground state = ', ansatz_obj.Get_BK_HF_state_in_OCC_basis())

JW ground state =  [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]
BK ground state =  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
# qubits_to_remove = Find_I_Z_indices_in_Hamiltonian(QubitHamiltonian, Hamilt.molecule.n_qubits)
# print('qubits only acted on by I or Z:', qubits_to_remove)


# input_state = ansatz_obj.Get_BK_HF_state_in_OCC_basis() if transformation=='BK' else ansatz_obj.Get_JW_HF_state_in_OCC_basis()
# # Remove_Z_terms_from_Hamiltonian
# NewQubitHamiltonian = Remove_Z_terms_from_Hamiltonian(
#                                                     QubitHamiltonian, 
#                                                     input_state,
#                                                     qubits_to_remove,
#                                                     check_reduction=True)
# NewQubitHamiltonian

In [ ]:
# qubitNo_re_label_dict, NewQubitHamiltonian_relabelled = Re_label_Hamiltonian(NewQubitHamiltonian)
# NewQubitHamiltonian_relabelled

In [11]:
NewQubitHamiltonian_relabelled=QubitHamiltonian

# Find what new FCI energy is
- here should be the same as true answer!

In [12]:
from openfermion import qubit_operator_sparse
from scipy.sparse import csc_matrix
new_Molecular_H_MATRIX =  csc_matrix(qubit_operator_sparse(NewQubitHamiltonian_relabelled))

# new_Molecular_H_MATRIX = np.flip(new_Molecular_H_MATRIX)

from scipy.sparse.linalg import eigs
try:
    eig_values, eig_vectors = eigs(new_Molecular_H_MATRIX)
except:
    from scipy.linalg import eig
    eig_values, eig_vectors = eig(new_Molecular_H_MATRIX.todense())
    
new_FCI_Energy = min(eig_values)


index = np.where(eig_values==new_FCI_Energy)[0][0]
ground_state_vector = eig_vectors[:, index]

print('new_FCI = ', new_FCI_Energy, 'VS old FCI:', fci_energy)
print(np.isclose(new_FCI_Energy, fci_energy))

new_FCI =  (-7.971184315565687-4.516472850092373e-16j) VS old FCI: -7.971184315565538
True


In [9]:
np.dot(ground_state_vector.conj().T, new_Molecular_H_MATRIX.dot(ground_state_vector))

(-7.971184315565682+4.4377750172547303e-16j)

# Ansatz Circuit

EMTPY!!!!!!!!!!

In [25]:
q_reg = QuantumRegister(12)
Ansatz_circuit = QuantumCircuit(q_reg)

In [27]:
# Ansatz_circuit.draw()

# Standard VQE

In [7]:
from tqdm.notebook import tqdm

In [28]:
standard_VQE_circuits, standard_I_term = Build_Standard_VQE_circuits_MEMORY_EFF(
                                                    NewQubitHamiltonian_relabelled, 
                                                    Ansatz_circuit,
                                                    q_reg)

Getting_standard_VQE_circuits: 631it [00:00, 2826.25it/s]


In [34]:
j=200

print(standard_VQE_circuits[j]['qubitOp'])

circ = QuantumCircuit.from_qasm_str(standard_VQE_circuits[j]['circuit'])
circ.draw()

(-6.573369273091304e-06+0j) [Y0 X1 Y3 Z5 Z6 X7 Z9 X10 X11]


┌─────────┐ ░ ┌─┐                                 
 q577_0: ┤ RX(π/2) ├─░─┤M├─────────────────────────────────
         └──┬───┬──┘ ░ └╥┘┌─┐                              
 q577_1: ───┤ H ├────░──╫─┤M├──────────────────────────────
            └───┘    ░  ║ └╥┘┌─┐                           
 q577_2: ────────────░──╫──╫─┤M├───────────────────────────
         ┌─────────┐ ░  ║  ║ └╥┘┌─┐                        
 q577_3: ┤ RX(π/2) ├─░──╫──╫──╫─┤M├────────────────────────
         └─────────┘ ░  ║  ║  ║ └╥┘┌─┐                     
 q577_4: ────────────░──╫──╫──╫──╫─┤M├─────────────────────
                     ░  ║  ║  ║  ║ └╥┘┌─┐                  
 q577_5: ────────────░──╫──╫──╫──╫──╫─┤M├──────────────────
                     ░  ║  ║  ║  ║  ║ └╥┘┌─┐               
 q577_6: ────────────░──╫──╫──╫──╫──╫──╫─┤M├───────────────
            ┌───┐    ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐            
 q577_7: ───┤ H ├────░──╫──╫──╫──╫──╫──╫──╫─┤M├────────────
            └───┘    ░  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐         
 q577_8: ────────────░──╫──╫──╫──╫──╫──╫──╫──╫─┤M├─────────
                     ░  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐      
 q577_9: ────────────░──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├──────
            ┌───┐    ░  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐   
q577_10: ───┤ H ├────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├───
            ├───┤    ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
q577_11: ───┤ H ├────░──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫──╫─┤M├
            └───┘    ░  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 12/═══════════════╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩══╩═
                        0  1  2  3  4  5  6  7  8  9  10 11

# Graph

In [13]:
from tqdm.notebook import tqdm

In [14]:
Hamiltonian_graph_obj = Openfermion_Hamiltonian_Graph(NewQubitHamiltonian_relabelled)

commutativity_flag = 'AC' ## <- defines relationship between sets!!!
plot_graph = False
Graph_colouring_strategy='largest_first'
anti_commuting_sets = Hamiltonian_graph_obj.Get_Clique_Cover_as_QubitOp(commutativity_flag, Graph_colouring_strategy=Graph_colouring_strategy, plot_graph=plot_graph)

anti_commuting_sets

Building Graph Edges: 100%|##########| 631/631 [00:18<00:00, 34.90it/s] 


{0: [(0.07828114149315563+0j) [Z9]],
 1: [(-4.142299396835114+0j) []],
 2: [(0.07828114149315557+0j) [Z3 Z5 Z7]],
 3: [(-0.22752612049271306+0j) [Z6],
  (0.004218766523001701+0j) [Z3 Z5 X6 Z7 X8],
  (0.0035571296112725677+0j) [Z3 Y4 X5 Y6 Y7 Y11],
  (0.010347236196697045+0j) [Y4 Z5 Y6],
  (-0.004733318899613399+0j) [X3 Y4 X5 Y6],
  (0.0025509806659133943+0j) [Y1 Y3 Y4 X5 Y6]],
 4: [(-0.2275261204927131+0j) [Z3 Z5 Z6 Z7],
  (0.004218766523001701+0j) [X6 X8 Z9],
  (0.005980395279375867+0j) [Z1 Y2 Z3 Y6],
  (0.004878950745533188+0j) [Z1 Y2 X3 Y6 Y7 Y11],
  (-0.0018744069004633774+0j) [X1 Y2 Y6],
  (-0.004733318899613399+0j) [Z1 Y2 Y3 Y5 Y6]],
 5: [(-0.22752612049271323+0j) [Z8 Z9],
  (0.004218766523001701+0j) [Y6 Y8 Z9],
  (0.005980395279375868+0j) [X2 X8 Z9],
  (-0.0047333188996134+0j) [Z1 Y2 Y3 Z4 X5 X8 Z9],
  (0.004878950745533189+0j) [Z1 Y2 X3 Y7 X8 Z10 X11],
  (0.0018744069004633785+0j) [Z0 Y1 Y2 X8 Z9]],
 6: [(-0.2275261204927132+0j) [Z8],
  (0.004218766523001701+0j) [Z3 Z5 Y6 Z7 Y8

# Seq Rot circuits

In [19]:
n_qubits= len(new_input_state)
rotation_reduction_check=False

Seq_Rot_VQE_circuits, Seq_Rot_I_term = Get_Seq_Rot_Unitary_Part_circuits_MEMORY_EFF(
                                anti_commuting_sets, 
                                Ansatz_circuit,
                                q_reg,
                                n_qubits,
                                S_index_dict=None, 
                                rotation_reduction_check=rotation_reduction_check)

Getting seq_rot VQE circuits: 100%|##########| 100/100 [00:03<00:00, 25.87it/s]


In [24]:
j=3

circ = QuantumCircuit.from_qasm_str(Seq_Rot_VQE_circuits[j]['circuit'])
circ.draw()

»
   q0_0: ─────────────────────────────────────────────────────────────»
                                                                      »
   q0_1: ──────────────────■──────────────────────────────────────────»
         ┌──────────────┐┌─┴─┐                                        »
   q0_2: ┤ U2(-π/2,π/2) ├┤ X ├────────────────────────────────────────»
         └──────────────┘└───┘                                        »
   q0_3: ───────■─────────────────────────────────────────────────────»
                │                                                     »
   q0_4: ───────┼─────────────────────────────────────────────────────»
              ┌─┴─┐                                                   »
   q0_5: ─────┤ X ├────────■──────────────────────────────────────────»
         ┌────┴───┴─────┐┌─┴─┐                                        »
   q0_6: ┤ U2(-π/2,π/2) ├┤ X ├──■─────────────────────────────────────»
         └──────────────┘└───┘┌─┴─┐                                   »
   q0_7: ─────────────────────┤ X ├──■─────────────────────────────■──»
           ┌─────────┐        └───┘┌─┴─┐                         ┌─┴─┐»
   q0_8: ──┤ U2(0,π) ├─────────────┤ X ├──■───────────────────■──┤ X ├»
           └─────────┘             └───┘┌─┴─┐┌─────────────┐┌─┴─┐└───┘»
   q0_9: ───────────────────────────────┤ X ├┤ U1(-3.1231) ├┤ X ├─────»
                                        └───┘└─────────────┘└───┘     »
  q0_10: ─────────────────────────────────────────────────────────────»
         ┌──────────────┐                                             »
  q0_11: ┤ U2(-π/2,π/2) ├─────────────────────────────────────────────»
         └──────────────┘                                             »
meas: 12/═════════════════════════════════════════════════════════════»
                                                                      »
«                                                                  »
«   q0_0: ─────────────────────────────────────────────────────────»
«                                                                  »
«   q0_1: ─────────────────────────────────────────────────────────»
«                                                                  »
«   q0_2: ─────────────────────────────────■───────────────────────»
«                                          │  ┌──────────────┐     »
«   q0_3: ───────────────────────■─────────┼──┤ U2(-π/2,π/2) ├─────»
«                                │         │  └──────────────┘     »
«   q0_4: ───────────────────────┼─────────┼───────────────────────»
«                              ┌─┴─┐     ┌─┴─┐                     »
«   q0_5: ─────────────■───────┤ X ├─────┤ X ├───────■─────────────»
«                    ┌─┴─┐┌────┴───┴────┐└───┘     ┌─┴─┐           »
«   q0_6: ─────■─────┤ X ├┤ U2(3π/2,-π) ├──────────┤ X ├────────■──»
«            ┌─┴─┐   └───┘└─────────────┘          └───┘      ┌─┴─┐»
«   q0_7: ───┤ X ├────────────────────────────────────────────┤ X ├»
«         ┌──┴───┴──┐                                         └───┘»
«   q0_8: ┤ U2(0,π) ├──────────────────────────────────────────────»
«         └─────────┘                                              »
«   q0_9: ─────────────────────────────────────────────────────────»
«                                                                  »
«  q0_10: ─────────────────────────────────────────────────────────»
«                                                                  »
«  q0_11: ─────────────────────────────────────────────────────────»
«                                                                  »
«meas: 12/═════════════════════════════════════════════════════════»
«                                                                  »
«                                                                       »
«   q0_0: ──────────────────────────────────────────────────────────────»
«                                                                       »
«   q0_1: ───────────────────────────────

In [ ]:
# n_qubits= len(new_input_state)
# rotation_reduction_check=False

# Seq_Rot_VQE_circuits, Seq_Rot_I_term = Get_Seq_Rot_Unitary_Part_circuits(
#                                 anti_commuting_sets, 
#                                 Ansatz_circuit,
#                                 q_reg,
#                                 n_qubits,
#                                 S_index_dict=None, 
#                                 rotation_reduction_check=rotation_reduction_check)

# LCU circuits

In [ ]:
# n_qubits= len(new_input_state)
# check_ansatz_state = False

# LCU_VQE_circuits, LCU_I_term= Get_LCU_Unitary_Part_circuits(anti_commuting_sets, ground_state_vector, n_qubits, 
#                                   N_index_dict=None,check_ansatz_state=check_ansatz_state)

In [18]:
n_qubits= len(new_input_state)


LCU_VQE_circuits, LCU_I_term=Get_LCU_Unitary_Part_circuits_MEMORY_EFF(
                                                                        anti_commuting_sets, 
                                                                        Ansatz_circuit,
                                                                        q_reg,
                                                                        n_qubits,
                                                                        N_index_dict=None)

Getting LCU VQE circuits: 100%|##########| 103/103 [1:14:51<00:00, 43.61s/it]


# Save experiment input

In [ ]:
list(NewQubitHamiltonian_relabelled)